In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import cv2
import PIL
import pydicom as dicom
from tensorflow.keras.layers import Dense,Conv2D, Dropout,Flatten, MaxPooling2D
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [2]:
nodCount = pd.read_excel("lidc-idri nodule counts (6-23-2015).xlsx",sheet_name = 'Sheet1')

In [3]:
nodCount.head(5)

,TCIA Patent ID,Total Number of Nodules*,Number of Nodules >=3mm**,Number of Nodules <3mm***,Unnamed: 4,Unnamed: 5
0,LIDC-IDRI-0001,4,1,3,NaN,NaN
1,LIDC-IDRI-0002,12,1,11,NaN,*total number of lesions that received either ...
2,LIDC-IDRI-0003,4,4,0,NaN,"**total number of lesions that received a ""nod..."
3,LIDC-IDRI-0004,4,1,3,NaN,"***total number of lesions that received a ""no..."
4,LIDC-IDRI-0005,9,3,6,NaN,NaN


In [4]:
nodCount.describe

<bound method NDFrame.describe of       TCIA Patent ID  Total Number of Nodules*   Number of Nodules >=3mm**  \
0     LIDC-IDRI-0001                          4                          1   
1     LIDC-IDRI-0002                         12                          1   
2     LIDC-IDRI-0003                          4                          4   
3     LIDC-IDRI-0004                          4                          1   
4     LIDC-IDRI-0005                          9                          3   
...              ...                        ...                        ...   
1014  LIDC-IDRI-1009                          2                          1   
1015  LIDC-IDRI-1010                         10                          1   
1016  LIDC-IDRI-1011                          4                          4   
1017  LIDC-IDRI-1012                          1                          1   
1018             NaN                       7371                       2669   

      Number of Nodules <3mm*

In [5]:
nodCount.columns

Index(['TCIA Patent ID', 'Total Number of Nodules* ',
       'Number of Nodules >=3mm**', 'Number of Nodules <3mm***', 'Unnamed: 4',
       'Unnamed: 5'],
      dtype='object')

In [6]:
nodCount.isnull().any()

TCIA Patent ID                True
Total Number of Nodules*     False
Number of Nodules >=3mm**    False
Number of Nodules <3mm***    False
Unnamed: 4                    True
Unnamed: 5                    True
dtype: bool

In [7]:
nodCount.isnull().sum()

TCIA Patent ID                  1
Total Number of Nodules*        0
Number of Nodules >=3mm**       0
Number of Nodules <3mm***       0
Unnamed: 4                   1019
Unnamed: 5                   1016
dtype: int64

In [8]:
nodCount.drop(['Unnamed: 4','Unnamed: 5'], inplace = True,axis=1)

In [9]:
nodCount = nodCount.dropna(axis=0)


In [10]:
nodCount.describe

<bound method NDFrame.describe of       TCIA Patent ID  Total Number of Nodules*   Number of Nodules >=3mm**  \
0     LIDC-IDRI-0001                          4                          1   
1     LIDC-IDRI-0002                         12                          1   
2     LIDC-IDRI-0003                          4                          4   
3     LIDC-IDRI-0004                          4                          1   
4     LIDC-IDRI-0005                          9                          3   
...              ...                        ...                        ...   
1013  LIDC-IDRI-1008                          7                          6   
1014  LIDC-IDRI-1009                          2                          1   
1015  LIDC-IDRI-1010                         10                          1   
1016  LIDC-IDRI-1011                          4                          4   
1017  LIDC-IDRI-1012                          1                          1   

      Number of Nodules <3mm*

In [11]:
patID = nodCount["TCIA Patent ID"]

In [12]:
totalNodules = nodCount["Total Number of Nodules* "]

In [13]:
noNodules = []
position= 0
for i in totalNodules:
    if i == 0:
        noNodules.append(patID[position])
    position+=1
    

In [14]:
print(noNodules)


['LIDC-IDRI-0306', 'LIDC-IDRI-0307', 'LIDC-IDRI-0322', 'LIDC-IDRI-0364', 'LIDC-IDRI-0441', 'LIDC-IDRI-0506', 'LIDC-IDRI-0540', 'LIDC-IDRI-0564', 'LIDC-IDRI-0573', 'LIDC-IDRI-0612', 'LIDC-IDRI-0668', 'LIDC-IDRI-0689', 'LIDC-IDRI-0716', 'LIDC-IDRI-0731', 'LIDC-IDRI-0760', 'LIDC-IDRI-0887', 'LIDC-IDRI-0900', 'LIDC-IDRI-0937', 'LIDC-IDRI-0975', 'LIDC-IDRI-0995']


In [15]:
pixels = []

In [16]:
hvNodules = []
position = 0
for i in totalNodules:
    if i != 0:
        hvNodules.append(patID[position])
    position+=1    
    

In [17]:
print(hvNodules)

['LIDC-IDRI-0001', 'LIDC-IDRI-0002', 'LIDC-IDRI-0003', 'LIDC-IDRI-0004', 'LIDC-IDRI-0005', 'LIDC-IDRI-0006', 'LIDC-IDRI-0007', 'LIDC-IDRI-0008', 'LIDC-IDRI-0009', 'LIDC-IDRI-0010', 'LIDC-IDRI-0011', 'LIDC-IDRI-0012', 'LIDC-IDRI-0013', 'LIDC-IDRI-0014', 'LIDC-IDRI-0015', 'LIDC-IDRI-0016', 'LIDC-IDRI-0017', 'LIDC-IDRI-0018', 'LIDC-IDRI-0019', 'LIDC-IDRI-0020', 'LIDC-IDRI-0021', 'LIDC-IDRI-0022', 'LIDC-IDRI-0023', 'LIDC-IDRI-0024', 'LIDC-IDRI-0025', 'LIDC-IDRI-0026', 'LIDC-IDRI-0027', 'LIDC-IDRI-0028', 'LIDC-IDRI-0029', 'LIDC-IDRI-0030', 'LIDC-IDRI-0031', 'LIDC-IDRI-0032', 'LIDC-IDRI-0033', 'LIDC-IDRI-0034', 'LIDC-IDRI-0035', 'LIDC-IDRI-0036', 'LIDC-IDRI-0037', 'LIDC-IDRI-0038', 'LIDC-IDRI-0039', 'LIDC-IDRI-0040', 'LIDC-IDRI-0041', 'LIDC-IDRI-0042', 'LIDC-IDRI-0043', 'LIDC-IDRI-0044', 'LIDC-IDRI-0045', 'LIDC-IDRI-0046', 'LIDC-IDRI-0047', 'LIDC-IDRI-0048', 'LIDC-IDRI-0049', 'LIDC-IDRI-0050', 'LIDC-IDRI-0051', 'LIDC-IDRI-0052', 'LIDC-IDRI-0053', 'LIDC-IDRI-0054', 'LIDC-IDRI-0055', 'LIDC-IDR

In [19]:
os.mkdir("Zero Nodules")

In [20]:
os.mkdir("Potentially Cancerous")


In [21]:
ORIGIN = "C:/Users/Hp/Documents/Data Science/LIDC/LIDC-IDRI"


In [24]:
folders = []
for f in os.listdir(ORIGIN):
    folders.append(f)
count = 0
for folder in folders:
    print(folder)
    if folder == hvNodules[count]:
        shutil.move((ORIGIN+'/'+folder),"Potentially Cancerous")
    else:
        
        shutil.move((ORIGIN+'/'+folder),'Zero Nodules')
    count+=1

LIDC-IDRI-0001
LIDC-IDRI-0002
LIDC-IDRI-0003
LIDC-IDRI-0004
LIDC-IDRI-0005
LIDC-IDRI-0006
LIDC-IDRI-0007
LIDC-IDRI-0008
LIDC-IDRI-0009
LIDC-IDRI-0010
LIDC-IDRI-0011
LIDC-IDRI-0012
LIDC-IDRI-0013
LIDC-IDRI-0014
LIDC-IDRI-0015
LIDC-IDRI-0016
LIDC-IDRI-0017
LIDC-IDRI-0018
LIDC-IDRI-0021
LIDC-IDRI-0022
LIDC-IDRI-0023
LIDC-IDRI-0024
LIDC-IDRI-0025
LIDC-IDRI-0026
LIDC-IDRI-0027
LIDC-IDRI-0028
LIDC-IDRI-0029
LIDC-IDRI-0030
LIDC-IDRI-0031
LIDC-IDRI-0032
LIDC-IDRI-0033
LIDC-IDRI-0034
LIDC-IDRI-0035
LIDC-IDRI-0036
LIDC-IDRI-0037
LIDC-IDRI-0038
LIDC-IDRI-0039
LIDC-IDRI-0040
LIDC-IDRI-0041
LIDC-IDRI-0042
LIDC-IDRI-0043
LIDC-IDRI-0044
LIDC-IDRI-0045
LIDC-IDRI-0046
LIDC-IDRI-0047
LIDC-IDRI-0048
LIDC-IDRI-0049
LIDC-IDRI-0050
LIDC-IDRI-0051
LIDC-IDRI-0052
LIDC-IDRI-0053
LIDC-IDRI-0054
LIDC-IDRI-0055


In [25]:
PNG = False

In [26]:
def convert_NoNodules():
    dcm_path = "Zero Nodules/"
    jpg_path = "Zero Nodules/Images"
    patients = os.listdir(dcm_path)
    images_path = []
    for i in os.listdir(dcm_path):
        images_path.append(os.path.join(dcm_path,i))
    for n,image in enumerate(images_path):
        ds = dicom.dcmread(image)
        pixel_array_numpy = ds.pixel_array
        image = image.replace('.dcm','.png')
        cv2.imwrite(image,pixel_array_numpy)
        
        if n%50 == 0:
            print('{} image converted'.format(n))

    pixels = pixel_array_numpy
    

In [68]:
def convert_hvNodules():
    dcm_path = "C:/Users/Hp/Documents/Data Science/LIDC/Potentially Cancerous"
    jpg_path = "C:/Users/Hp/Documents/Data Science/LIDC/Potentially Cancerous/Images"
    patients = os.listdir(dcm_path)
    images_path = []
    #os.mkdir(jpg_path)
    X =[]
    y = []
    for i in patients:
      ##  images_path.append(os.path.join(dcm_path,i))
        images_path.append((dcm_path+"/"+i))
    for patient in images_path:
        print(patient)
        for img in os.listdir(patient):
            print(img)
            ds = dicom.dcmread((patient+'/'+img))
            pixel_array_numpy = ds.pixel_array
            X.append(pixel_array_numpy)
            y.append("Cancerous")
            img = img.replace('.dcm','.png')
            os.chdir(jpg_path)
            cv2.imwrite(img,pixel_array_numpy)
            pixel_array = cv2.imread(img)
            print("done")

    pixels = pixel_array_numpy
        
        #if n%50 == 0:
           # print('{} image converted'.format(n))

    
    

In [124]:
from tensorflow.keras import backend as K
print(K.image_data_format())

channels_last


In [133]:
classifier = Sequential()
#classifier.add(Flatten())
classifier.add(Conv2D(32,kernel_size=(3,3),input_shape=(512,512,3),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dense(256,activation='relu'))
classifier.add(Dense(2,activation = 'softmax'))

classifier.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [71]:
convert_hvNodules()

C:/Users/Hp/Documents/Data Science/LIDC/Potentially Cancerous/Images
C:/Users/Hp/Documents/Data Science/LIDC/Potentially Cancerous/LIDC-IDRI-0001
1-001.dcm
done
1-002.dcm
done
1-003.dcm
done
1-004.dcm
done
1-005.dcm
done
1-006.dcm
done
1-007.dcm
done
1-008.dcm
done
1-009.dcm
done
1-010.dcm
done
1-011.dcm
done
1-012.dcm
done
1-013.dcm
done
1-014.dcm
done
1-015.dcm
done
1-016.dcm
done
1-017.dcm
done
1-018.dcm
done
1-019.dcm
done
1-020.dcm
done
1-021.dcm
done
1-022.dcm
done
1-023.dcm
done
1-024.dcm
done
1-025.dcm
done
1-026.dcm
done
1-027.dcm
done
1-028.dcm
done
1-029.dcm
done
1-030.dcm
done
1-031.dcm
done
1-032.dcm
done
1-033.dcm
done
1-034.dcm
done
1-035.dcm
done
1-036.dcm
done
1-037.dcm
done
1-038.dcm
done
1-039.dcm
done
1-040.dcm
done
1-041.dcm
done
1-042.dcm
done
1-043.dcm
done
1-044.dcm
done
1-045.dcm
done
1-046.dcm
done
1-047.dcm
done
1-048.dcm
done
1-049.dcm
done
1-050.dcm
done
1-051.dcm
done
1-052.dcm
done
1-053.dcm
done
1-054.dcm
done
1-055.dcm
done
1-056.dcm
done
1-057.dcm
done

InvalidDicomError: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.

In [37]:
#convert_NoNodules()

In [38]:
CATEGORY = ["Potentially Cancerous", "No Nodules"]


In [94]:
for i in os.listdir('C:/Users/Hp/Documents/Data Science/LIDC/Potentially Cancerous/Images'):
    img_array = cv2.imread(('C:/Users/Hp/Documents/Data Science/LIDC/Potentially Cancerous/Images/'+i))
    X.append(img_array)
    y.append(1)


In [96]:
print(X)

[array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8), array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,

In [97]:
X = np.array(X)
X = X/255

In [93]:
X = []
y = []

In [76]:
for i in os.listdir('C:/Users/Hp/Documents/Data Science/LIDC/Zero Nodules/Images'):
    img_array = cv2.imread(('C:/Users/Hp/Documents/Data Science/LIDC/Zero Nodules/Images/'+i))
    X.append(img_array)
    y.append("Not Cancerous")
    

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Users/Hp/Documents/Data Science/LIDC/Zero Nodules/Images'

In [ ]:
x_train, x_test,y_train,y_test = train_test_split(X,y,random_state = 0)

In [87]:
count = 0
for x in X:
    X[count] = x/255
    print(X[count])
    count+=1

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  .

  [0. 0. 0.]]]
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
 

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  .

  [0. 0. 0.]]]
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
 

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  .

In [99]:
y = np.array(y)

In [132]:
classifier.fit(X,y,batch_size=32, epochs=40, validation_split=0.1)

Epoch 1/40


InvalidArgumentError:  Incompatible shapes: [32,1] vs. [32,255,255]
	 [[node Equal (defined at <ipython-input-132-c1fbb4d566fe>:1) ]] [Op:__inference_train_function_2099]

Function call stack:
train_function


In [59]:
data = dicom.dcmread("C:/Users/Hp/Documents/Data Science/LIDC/Potentially Cancerous/LIDC-IDRI-0001/1-002.dcm")

In [66]:
print(data)

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 206
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.6.1.4.1.14519.5.2.1.6279.6001.512235483218154065970649917292
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.3.6.1.4.1.22213.1.143
(0002, 0013) Implementation Version Name         SH: '0.5'
(0002, 0016) Source Application Entity Title     AE: 'POSDA'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL']
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.14519.5.2.1.6279.6001.51223

In [104]:
i = (data.pixel_array.shape)

In [106]:
print(i)

(512, 512)
